In [ ]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
# Cargar el conjunto de datos
df_EmpAttrition = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataSets/empleadosRETO.csv')


In [ ]:
# Eliminar las columnas especificadas que no tienen relación con la salida
columns_to_drop = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours', 'PerformanceRating', 'JobLevel']
df_EmpAtriClean = df_EmpAttrition.drop(columns=columns_to_drop)

# Evaluar las columnas restantes para obtener un resumen estadístico
#column_info = df_EmpAtriClean.describe(include='all')
#print(column_info)


In [ ]:
# Convertir la columna 'HiringDate' a un formato de fecha
df_EmpAtriClean['HiringDate'] = pd.to_datetime(df_EmpAtriClean['HiringDate'], errors='coerce')

invalid_dates = df_EmpAtriClean[df_EmpAtriClean['HiringDate'].isna()]
#print("Fechas inválidas:")
#print(invalid_dates)

df_EmpAtriClean = df_EmpAtriClean.dropna(subset=['HiringDate'])
#print(df_EmpAtriClean['HiringDate'].dtype)  # Debe mostrar "datetime64[ns]"

# Crear la columna 'Year' extrayendo el año de la fecha de contratación
df_EmpAtriClean['Year'] = df_EmpAtriClean['HiringDate'].dt.year.astype(int)

# Crear la columna 'YearsAtCompany' calculando los años desde la fecha de contratación hasta 2018
df_EmpAtriClean['YearsAtCompany'] = 2018 - df_EmpAtriClean['Year']

#print(df_EmpAtriClean[['HiringDate', 'Year', 'YearsAtCompany']].head())


In [ ]:
# Renombrar la columna 'DistanceFromHome' a 'DistanceFromHome_km'
df_EmpAtriClean.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace=True)

# Crear una nueva columna 'DistanceFromHome' eliminando las letras 'km' y convirtiendo a entero
df_EmpAtriClean['DistanceFromHome'] = df_EmpAtriClean['DistanceFromHome_km'].str.replace(' km', '').astype(int)

#print(df_EmpAtriClean[['DistanceFromHome_km', 'DistanceFromHome']].head())

In [ ]:
# Eliminar las columnas 'Year', 'HiringDate', y 'DistanceFromHome_km'
columns_to_remove = ['Year', 'HiringDate', 'DistanceFromHome_km']
df_EmpAtriClean.drop(columns=columns_to_remove, inplace=True)

#print(df_EmpAtriClean.head())

In [ ]:
# Calcular el ingreso promedio por departamento
df_SueldoPromedioDepto = df_EmpAtriClean.groupby('Department')['MonthlyIncome'].mean().reset_index()

# Renombrar la columna con el sueldo promedio
df_SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace=True)

df_SueldoPromedioDepto

In [ ]:
# Escalar la variable 'MonthlyIncome'
scaler = MinMaxScaler()
df_EmpAtriClean['MonthlyIncomeScaled'] = scaler.fit_transform(df_EmpAtriClean[['MonthlyIncome']])

#df_EmpAtriClean[['MonthlyIncome', 'MonthlyIncomeScaled']].head()

In [ ]:
## Convertir la variable 'Attrition' a numérica: 'Yes' -> 1, 'No' -> 0
df_EmpAtriClean['Attrition'] = df_EmpAtriClean['Attrition'].map({'Yes': 1, 'No': 0})

# Convertir las otras variables categóricas a numéricas
categorical_columns = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']
df_EmpAtriClean = pd.get_dummies(df_EmpAtriClean, columns=categorical_columns, drop_first=True)

# Asegurarse de que todas las columnas son numéricas
df_EmpAtriClean = df_EmpAtriClean.apply(pd.to_numeric, errors='coerce')

In [ ]:
# Calcular la correlación entre las variables y 'Attrition'
correlation_matrix = df_EmpAtriClean.corr()

# Extraer las correlaciones de 'Attrition' con todas las demás variables
attrition_correlation = correlation_matrix['Attrition'].sort_values(ascending=False)

attrition_correlation

In [ ]:
# Filtrar las variables que tienen una correlación mayor o igual a 0.1
selected_columns = attrition_correlation[attrition_correlation.abs() >= 0.1].index

# Crear el nuevo DataFrame EmpleadosAttritionFinal con las columnas seleccionadas
EmpleadosAttritionFinal = df_EmpAtriClean[selected_columns]

#EmpleadosAttritionFinal.head()

In [ ]:
# Aplicar PCA
pca = PCA()
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal)

EmpleadosAttritionPCA[:5]

In [ ]:
# Calcular la cantidad de componentes necesarios para explicar el 80% de la varianza
explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
n_components = (explained_variance_ratio >= 0.80).argmax() + 1

# Seleccionar los primeros n_components
selected_components = EmpleadosAttritionPCA[:, :n_components]

# Crear un DataFrame con los componentes seleccionados
component_names = [f'C{i}' for i in range(n_components)]
components_df = pd.DataFrame(selected_components, columns=component_names)

# Agregar los componentes al DataFrame EmpleadosAttritionFinal
EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**components_df)

EmpleadosAttritionFinal.head()

#print(f"Varianza explicada por cada componente: {explained_variance_ratio}")

Varianza explicada por cada componente: [0.99999448 0.99999746 0.99999913 0.99999968 0.99999985 0.9999999
 0.99999995 0.99999998 0.99999998 0.99999999 0.99999999 1.
 1.         1.         1.        ]


In [ ]:
# Reordenar las columnas
columns_order = [col for col in EmpleadosAttritionFinal.columns if col != 'Attrition'] + ['Attrition']
EmpleadosAttritionFinal = EmpleadosAttritionFinal[columns_order]

EmpleadosAttritionFinal.to_csv('/content/drive/MyDrive/Colab Notebooks/DataSets/EmpleadosAttritionFinal.csv', index=False)